# An idea to improve performance of local dimensionality determination.

To find what the local dimension is, we can often use MDS on neighborhoods of points. If we performed MDS to find the first $k$ eigenvalues and eigenvectors of the similarity matrix in one neighborhood, then the information can be retained and used for other neighborhoods that share the same points. In the example below, we suppose a single point is being swapped. In this case, we can decompose the (symmetric) similarity matrix as
$$
M = 
U \Sigma U^T
$$
where the first $k$ columns of $U$ are the largest orthonormal eigenvectors of $M$ and the remaining columns complete an orthonormal basis.
Here
$$
\Sigma = \begin{pmatrix}
\Sigma_k && 0 \\
0 && \cdot
\end{pmatrix}
$$
where $\Sigma_k$ is a diagonal matrix with the first $k$ eigenvectors on its diagonal.

If a point in the neighborhood is swapped to a new point, then a column and a row of $M$ will change in some unknown way forming a new matrix $\tilde{M}$. If this change is small enough, then the first $k$ eigenvalues of $\tilde{M}$ can be computed more easily using $U^T \tilde{M} U$, since it is some perturbation of $\Sigma$. 

In [128]:
import sys

## numerical librarires
import numpy as np
from numpy import linalg as la
from scipy.linalg import orth
import random

## plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [165]:
from copy import copy

In [166]:
def print_mat(M,dec = 3):
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            sys.stdout.write(str(round(M[i,j],dec)) + ", ")
            
        print 

In [167]:
k = 2
n = 4
M = np.asmatrix(np.reshape([random.gauss(0,1) for _ in range(n**2)],(n,n)))
M = M * M.T

In [168]:
print_mat(M)

1.846, -0.276, -0.018, -0.455, 
-0.276, 0.053, 0.054, 0.304, 
-0.018, 0.054, 1.768, 3.397, 
-0.455, 0.304, 3.397, 10.729, 


In [169]:
vals,vecs = la.eig(M)

In [183]:
## Confirm this is M

print_mat(vecs*np.diag(vals)*vecs.T)

1.846, -0.276, -0.018, -0.455, 
-0.276, 0.053, 0.054, 0.304, 
-0.018, 0.054, 1.768, 3.397, 
-0.455, 0.304, 3.397, 10.729, 


In [184]:
trunc_vecs = np.asmatrix(np.eye(n))
for i in range(n):
    for j in range(k):
        trunc_vecs[i,j] = vecs[i,j]

In [185]:
print_mat(trunc_vecs)

0.044, -0.983, 0.0, 0.0, 
-0.027, 0.144, 0.0, 0.0, 
-0.318, -0.115, 1.0, 0.0, 
-0.947, -0.011, 0.0, 1.0, 


In [186]:
## build an orthonormal basis
q,r = la.qr(trunc_vecs)
print_mat(q)

-0.044, 0.983, 0.105, 0.145, 
0.027, -0.144, -0.009, 0.989, 
0.318, 0.115, -0.941, -0.0, 
0.947, 0.011, 0.321, -0.021, 


In [192]:
Sigma = q.T*M*q

In [193]:
print_mat(Sigma)

11.899, 0.0, 0.0, 0.0, 
0.0, 1.879, 0.0, -0.0, 
0.0, 0.0, 0.612, 0.022, 
0.0, -0.0, 0.022, 0.006, 


In [187]:
## The eigenvalue matrix. Above we can see the first $k$ eigenvalues.
print_mat(np.diag(vals))

11.899, 0.0, 0.0, 0.0, 
0.0, 1.879, 0.0, 0.0, 
0.0, 0.0, 0.006, 0.0, 
0.0, 0.0, 0.0, 0.612, 


In [178]:
M_tilde = copy(M)

In [188]:
## some small perturbation, if a point is replaced
for i in range(M.shape[0]-1):
    M_tilde[i,-1] -= 0.1
    M_tilde[-1,i] -= 0.1

In [194]:
## notice how close this matrix is to $Sigma$.
print_mat(q.T*M_tilde*q)

11.785, -0.181, 0.141, -0.213, 
-0.181, 1.875, -0.059, 0.001, 
0.141, -0.059, 0.72, -0.054, 
-0.213, 0.001, -0.054, 0.016, 
